In [ ]:
import glob
import json
import re
import os
from os import path


In [ ]:
gpu = False
import tensorflow as tf
if gpu:
    from keras.backend.tensorflow_backend import set_session
    physical_devices = tf.config.list_physical_devices('GPU') 
    try: 
      tf.config.experimental.set_memory_growth(physical_devices[0], True) 
    except: 
      # Invalid device or cannot modify virtual devices once initialized. 
      pass # dynamically grow GPU memory 


In [ ]:
def clean_images(basepath):
    count = 0
    images = glob.glob(f'{basepath}/**/*.jpg')
    for image in images:
        os.remove(image)
        count += 1
    print("Cleaned ", count, " images.")
    audioFiles = glob.glob(f'{basepath}/**/*.wav')
    for wav in audioFiles:
        os.remove(wav)

In [ ]:
clean_images("train")
clean_images("test")
clean_images("validation")

In [ ]:
def fileExists(basepath, file):
    return path.exists(f'{basepath}/{file}')

In [ ]:
import numpy as np
from scipy.spatial.distance import euclidean
import librosa

from fastdtw import fastdtw

def isAudioFake(basepath, fakeMP4, originalMP4):
    if fileExists(basepath, fakeMP4) and fileExists(basepath, originalMP4):
        fakeAudio = re.sub(r'(\.mp4$)', '.wav', fakeMP4)
        if not fileExists(basepath, fakeAudio):
            !ffmpeg -i {basepath}/{fakeMP4} -ab 160k -ac 2 -ar 44100 -vn {basepath}/{fakeAudio}
        originalAudio = re.sub(r'(\.mp4$)', '.wav', originalMP4)
        if not fileExists(basepath, originalAudio):
            !ffmpeg -i {basepath}/{originalMP4} -ab 160k -ac 2 -ar 44100 -vn {basepath}/{originalAudio}
    
        y1, sr1 = librosa.load(f'{basepath}/{fakeAudio}')
        y2, sr2 = librosa.load(f'{basepath}/{originalAudio}')
        mfcc1 = librosa.feature.mfcc(y1,sr1)   #Computing MFCC values
        mfcc2 = librosa.feature.mfcc(y2, sr2)
        distance, path = fastdtw(mfcc1.T, mfcc2.T, dist=euclidean)
        print("distance = ", distance)
        return distance > 0
    else:
        return False


In [ ]:
def process_movie(basepath, moviePath, label, split):
    if fileExists(basepath, moviePath):
        print("Processing movie, ", moviePath, " with label, ", label, " and split, ", split)
        movie = re.sub(r'(\.mp4$)', '', moviePath)
        !ffmpeg -i {basepath}/{moviePath} -r 1/1 {split}/{label}/{movie}%08d.jpg
        return 1
    return 0

In [ ]:
#metadataFiles = glob.glob('G:/deepfake/**/**/metadata.json')
metadataFiles = glob.glob('movies/**/metadata.json')
MAX_MOVIES=120

moviesProcessed = 0
testSamplingRate = 5  # every 5th movie, put into test, every sixth movie put into validation

def processMetadataFile(metadataFile, moviesProcessed):
    numRealMovies = 0
    numFakeMovies = 0
    numAudioFakes = 0
    originalsProcessed = 0
    audioFakes = []

    for metadata in metadataFile:
        basepath=re.sub(r'(metadata.json$)', '', metadata)[:-1]
        with open(metadata) as f:        
            data = json.load(f)
            for key in data:
                if moviesProcessed > MAX_MOVIES:
                    print("moviesProcessed > MAX")
                    break
                label = data[key]['label']
                if moviesProcessed % testSamplingRate == 0:
                    split = "test"
                elif moviesProcessed % testSamplingRate == 1:
                    split = "validation"
                else:
                    split = "train"
                if label == "FAKE" and 2 * numFakeMovies > numRealMovies:
                    print("Skipping because we need balanced fake and real videos")
                elif label == "FAKE":
                    original = data[key]['original']
                    
                    if isAudioFake(basepath, key, original):
                        numAudioFakes += 1
                        audioFakes.append(f'{basepath}/{key}')
                    else:
                        processed = process_movie(basepath, key, label, split)
                        moviesProcessed += processed
                        numFakeMovies += processed

                        processed = process_movie(basepath, original, "REAL", split)
                        moviesProcessed += processed
                        numRealMovies += processed
                        originalsProcessed += processed

                else:
                    numRealMovies += 1
                    processed = process_movie(basepath, key, label, split)

                    moviesProcessed += processed
    print("Processed originals", originalsProcessed, " videos for train.")
    print("Detected and skipped ", numAudioFakes, " fake videos with fake audio.")

    return moviesProcessed, audioFakes
      
(moviesProcessed, audioFakes) = processMetadataFile(metadataFiles, moviesProcessed)
    

In [ ]:
print("Processed ", moviesProcessed, " videos for train.")
print("audioFakes are ", audioFakes)

In [ ]:
from detect_face import crop_face_save_jpg

In [ ]:
trainImages = glob.glob('train/**/*.jpg')


In [ ]:
# Find the faces in each image, and crop it out
# If the face can't be detected, then we remove the image from dataset
for image in trainImages:
    crop_face_save_jpg(image)

In [ ]:
testImages = glob.glob('test/**/*.jpg')
for image in testImages:
    crop_face_save_jpg(image)

In [ ]:
testImages = glob.glob('validation/**/*.jpg')
for image in testImages:
    crop_face_save_jpg(image)

In [ ]:
print("Finished preprocessing movies")

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
import pandas as pd
from pickle import dump

def extract_features(filepath):
    """takes the cropped faces and extracts features from them. takes the file path of the face data"""
    # load model
    model = VGG16()
    # remove the output layer
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    features = np.zeros((1, 4096))
    filepath = os.path.join(filepath,'*.jpg')
    faces = glob.glob(filepath)
    for face in faces:
        # load an image from file
        image = load_img(face, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        #print(image.shape)
        #prepare the image for the VGG model
        image = preprocess_input(image)
        # get extracted features
        features = np.append(features,  model.predict(image), axis = 0)
        #print(features.shape)
        # save to file
        #dump(features, f)
    #save to pd dataframe
    features = np.delete(features, 0, 0)
    return pd.DataFrame(features)

In [ ]:
model = VGG16()

In [ ]:
train_fake_df = extract_features('train/fake/')
train_real_df = extract_features('train/real/')

test_real_df = extract_features('test/real/')
test_fake_df = extract_features('test/fake/')

In [ ]:
validation_real_df = extract_features('validation/real/')
validation_fake_df = extract_features('validation/fake/')

In [ ]:
# write extracted features to disk
train_fake_df.to_csv('train_fake.csv')
train_real_df.to_csv('train_real.csv')
test_fake_df.to_csv('test_fake.csv')
test_real_df.to_csv('test_real.csv')

In [ ]:
validation_fake_df.to_csv('validation_fake.csv')
validation_real_df.to_csv('validation_real.csv')

In [ ]:
#model = VGG16()

In [ ]:
#model.layers.pop()
#model.layers.pop()
#model.layers.pop()

#model.summary()